# 2 - Machine Learning for Optical Network Systems

Welcome to the second part of the practical session of the Workshop __Machine Learning for Optical Network Systems!__
In this exercise you will:
 - Implement a prediction model based on the Support Vector Machines (SVM) algorithm in order to predict the Quality of Transmission in terms of the Optical Signal-to-Noise Ratio (OSNR) of an optical lightpath before establishement.
 - Use your model to build a multi-classification system that will determine the appropriate modulation format to be used for transmission give the predicted OSNR.
 - Use practical statistical tools to evaluate the performance of your SVM-based classification system.

__Let's go!__

## Problem description

Technologies such as Bandwidth Variable Transceivers (BVTs) are enabling the optimise usage of spectrum resources in an emerging area commonly referred to as Elastic Optical Networks (EONs). In the context of this type of networks, a control/management system should be able to dynamically reconfigure the modulation format and phase of a transceiver for spectrum and wavelength allocation. In recent years, efforts have been put together to implement cognitive performance as part of the control systems on top of optical networks. In this example, you are going to build a multi-classification system that will allow you to determine the appropriate modulation format to use for a BVT, given a lightpath request.

This exercise is based on the research paper Active Wavelength Load as a Feature for QoT Estimation Based on Support Vector Machine, which you can find in this folder (Go to File -> Open -> and click on the directory publications/ and file diaz2019active.pdf). Please refer to the paper for the description of the data.


## Getting started

### 1 - Import the required libraries

Execute the cell below so you can use the important libraries that will allow you to do the exercies. If you have doubts about the Scikit-learn framework please refer to the [documentation](https://scikit-learn.org/stable/documentation.html).

In [ ]:
import file_reader as fr
import numpy as np
import plot_statistics as ps
from sklearn import svm, metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

### 2 - Load the dataset

The dataset you are about to load is already split respecting a 80/20 ratio for training and testing set respectively. Run the cell below.

In [ ]:
training_set_file = 'dataset/trainingset.csv'
testing_set_file = 'dataset/testset.csv'

# Retrieve and format data for training set
X_train, y_train = fr.FileReader.read_array_three_class(fr.FileReader(), training_set_file)

# Retrieve and format data for testing set
X_test, y_test = fr.FileReader.read_array_three_class(fr.FileReader(), testing_set_file)

### 3 - Building the SVM model

Now you are going to create an instance of your SVM model that we will be using later. The cell below will create an Support Vector Classifier (SVC) object and will use hyperparameters of $ C = 100 $, $ gamma = 0.0001 $, and Radial Basis Function as the kernel function. If you have any doubts about the impact of these hyperparameters please refer to the [documentation](https://scikit-learn.org/stable/modules/svm.html#svm-classification). When you are done with this exercise, come back and play around different values for these or other hyperparameters to see the different effects.

In [ ]:
# Set the parameters
parameters = {'kernel':['rbf'], 'C':[10], 'gamma':[0.0001]}

# Instantiate SVC model
svc = svm.SVC()

# Display the SVC model
print(svc)

One of the strongest features of Scikit-learn is the GridSearchCV tool from model_selection, which allows you to search the best hyperparameters for a given model, allows you to indicate the number of CPU cores you want your algorithm to use, and in allows you to use cross-validation when training your data. If you are not familiar with neither the tool nor the usage of cross-validation, please refer to the following resources: [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [Cross-Validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)). In the cell below, you will create a SVC classifier with the parameters you have declared above, and you will use 10 CPU cores and use a cross-validation of 5-fold.

In [ ]:
classifier = GridSearchCV(svc, parameters, n_jobs=10, cv=5)

print(classifier)

__Congratulations!__ You have built an SVC classifier that is ready to learn.

### 4 - Fitting the data into your model

Now what you need to do is to fit your data into the model so it can learn about it, and then use it to predict the QoT of new unestablished lightpaths. This may take a few seconds to run.

In [ ]:
# Reshape y_train so you can fit the data with classifier.fit()
y_train = np.argmax(y_train,axis=1)

# Fit the data into the model and evaluate with the testing dataset
y_score = classifier.fit(X_train,y_train).decision_function(X_test)

print(y_score)

### 5 - Evaluating the performance

Once you have fit your data into your model, let's evaluate how well it does for predicting and classifying the unestablished lightpaths from the testing dataset.
With Scikit-learn there are multiple [metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) you can use to do so, in this example you will be using the [Receiver Operating Characteristic (ROC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve and the [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix). Also, you will be generating a classification report with the [precision](https://en.wikipedia.org/wiki/Precision_(statistics)), [recall](https://en.wikipedia.org/wiki/Precision_and_recall) and [F1-score](https://en.wikipedia.org/wiki/F1_score) of your classifier for the multiple OSNR-classes. With the cell below you will plot these metrics; the code to do so has been provided to you.

In [ ]:
ps_obj = ps.PlotStats()

fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = y_test.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

ps_obj.plot_roc_curve_multi(fpr, tpr, roc_auc, n_classes)

predicted = classifier.predict(X_test)
y_test_rs = np.argmax(y_test, axis=1)

cnf_matrix = metrics.confusion_matrix(y_test_rs, predicted)

ps_obj.plot_confusion_matrix(cnf_matrix, 
                            classes=[0, 1, 2, 3],
                            normalize=True, 
                            title='Confusion matrix QoT-E ')

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test_rs, predicted)))
      

As you can see from the report, you have an SVC model that has a good F1-score for the first and fourth classes (82% and 66% respectively), but does poorly in the other classes. In the next section of this practical session we will try to improve these.